# **Reto I**

### 1. Datasets

Los datos de origen constan de dos archivos csv con la misma estructura y tipo de columnas.

* trade_details: dataset original con datos reales de operaciones financieras.
* trade_details_snapshot: copia de seguridad por posibles perdidas de datos.

### 2. Columnas y significado:

* mfamily: indica la familia de operaciones a la que pertenece.
* mgroup: indica el grupo de operaciones dentro de la familia.
* mtype: indica el tipo de operación dentro del grupo.
* origin_trade_number: indica el número de la operación de trading (la misma operación puede tener varios números de trading).
* origin_contract_number: indica el número de contrato de la operación (igual para todas las operaciones que pertenecen al mismo contrato).
* maturity: fecha de finalización del contrato de cada operación.

### 3. Descripción del problema:

En estos datasets se encuentran varias operaciones financieras de distinto tipo, que diferenciaremos mediante los distintos valores de las columnas mfamily, mgroup y mtype.

Existe un cierto tipo de operaciones especiales, llamadas FXSwaps. Estas pueden ser diferenciadas por medio de los siguientes valores:

**mfamily = CURR** \
**mgroup = FXD** \
**mtype = SWLEG**

Podemos ver en nuestro dataset que estas operaciones aparecen duplicadas, es decir, con el mismo **origin_contract_number** aunque distinto **origin_trade_number**. De estas operaciones duplicadas en origen, queremos obtener solo una de ellas.

La forma para decidir cuál de las operaciones nos interesa obtener es mediante la columna *maturity*. De ambas operaciones de trading (distinto origin_trade_number) para un mismo contrato (origin_contract_number), queremos obtener solo la *long leg*, es decir, la que tiene una mayor fecha de vencimiento (fecha más actual de la columna maturity).

Existe un cierto problema en nuestro dataset trade_details que tendremos que solucionar. Podemos ver que para algunas operaciones el campo maturity vendrá como *null*, es decir, sin informar. En estos casos, deberemos buscar esa operacion en el dataset trade_details_snapshot y el respectivo campo maturity para poder saber cuál de las dos operaciones es la *long leg* y filtrar la *short leg* 

**NOTA: Si se quiere conocer más el significado de estas operaciones financieras: https://es.wikipedia.org/wiki/Swap_(finanzas)**

### 4. Reto:

* Obtener un dataframe final donde tengamos todas las operaciones originales excepto los short leg de los contratos tipo FXSwap.
* Aunque usemos el valor de la columna maturity del dataset trade_details_snapshot en los casos que venga en la trade_details a *null*, en el dataframe final deberá venir con el valor original de trade_details.
* Hacerlo de la manera más eficiente posible a nivel computacional.

### Inicialización de SparkSession:

In [1]:
import org.apache.spark.sql.SparkSession

val spark = SparkSession.builder()
                        .appName("Reto 1")
                        .master("local")
                        .getOrCreate()

Intitializing Scala interpreter ...

Spark Web UI available at http://L2111027.bosonit.local:4040
SparkContext available as 'sc' (version = 3.1.2, master = local[*], app id = local-1641378471947)
SparkSession available as 'spark'


import org.apache.spark.sql.SparkSession
spark: org.apache.spark.sql.SparkSession = org.apache.spark.sql.SparkSession@38dc2c36


### Carga de CSV

In [2]:
val trade_details = spark.read.format("csv")
                              .option("header", "true")
                              .option("delimiter", ";")
                              .load("./reto1/trade_details.csv")

val trade_details_snapshot = spark.read.format("csv")
                                       .option("header", "true")
                                       .option("delimiter", ";")
                                       .load("./reto1/trade_details_snapshot.csv")

trade_details: org.apache.spark.sql.DataFrame = [mfamily: string, mgroup: string ... 4 more fields]
trade_details_snapshot: org.apache.spark.sql.DataFrame = [mfamily: string, mgroup: string ... 4 more fields]


In [3]:
// Observando un poco el dataset

trade_details.show(5,false)

trade_details.printSchema()

trade_details.select(count("*")).show()

+-------+------+-----+-------------------+----------------------+----------+
|mfamily|mgroup|mtype|origin_trade_number|origin_contract_number|maturity  |
+-------+------+-----+-------------------+----------------------+----------+
|IRD    |BOND  |null |316391872          |678876251             |2021-09-22|
|CURR   |FXD   |FXD  |32734782           |54853428              |2021-09-22|
|IRD    |LN_BR |null |1111               |2222                  |2022-10-06|
|IRD    |IRS   |null |2222222            |2222222               |2024-10-15|
|SCF    |SCF   |SCF  |3815982            |3672136               |NULL      |
+-------+------+-----+-------------------+----------------------+----------+
only showing top 5 rows

root
 |-- mfamily: string (nullable = true)
 |-- mgroup: string (nullable = true)
 |-- mtype: string (nullable = true)
 |-- origin_trade_number: string (nullable = true)
 |-- origin_contract_number: string (nullable = true)
 |-- maturity: string (nullable = true)

+--------+
|count

In [4]:
// PRIMERO RELLENA TODOS LOS HUEVOS VACÍOS DE MATURITY (en una nueva tabla porque necesitamos conservar los valores null)

// Une el dataset trade_details y trade_details snapshot

val trade_join = trade_details_snapshot.as("b").join(trade_details.as("a"),$"a.origin_trade_number"===
$"b.origin_trade_number","inner")

// De el join que acabamos de hacer saca sólo las columnas que nos interesan
val trade_join_subset = trade_join.select($"a.*",$"b.maturity")

// Crea una columna nueva que almacene el valor maturity con el que realmente vamos a trabajar
val trade_join_maturity = trade_join_subset.withColumn("maturityReal", when($"a.maturity" === "NULL",$"b.maturity").otherwise($"a.maturity")).drop($"b.maturity")

trade_join_maturity.show()


+-------+------+-----+-------------------+----------------------+----------+------------+
|mfamily|mgroup|mtype|origin_trade_number|origin_contract_number|  maturity|maturityReal|
+-------+------+-----+-------------------+----------------------+----------+------------+
|    IRD|  BOND| null|          316391872|             678876251|2021-09-22|  2021-09-22|
|   CURR|   FXD|  FXD|           32734782|              54853428|2021-09-22|  2021-09-22|
|    IRD| LN_BR| null|               1111|                  2222|2022-10-06|  2022-10-06|
|    IRD|   IRS| null|            2222222|               2222222|2024-10-15|  2024-10-15|
|    SCF|   SCF|  SCF|            3815982|               3672136|      NULL|  2021-05-04|
|    IRD| LN_BR| null|           14596583|              13774383|2020-12-29|  2020-12-29|
|    IRD|   IRS| null|           18343978|              17356077|2024-10-15|  2024-10-15|
|    IRD| LN_BR| null|           19203839|              18176215|2022-10-06|  2022-10-06|
|    IRD| 

trade_join: org.apache.spark.sql.DataFrame = [mfamily: string, mgroup: string ... 10 more fields]
trade_join_subset: org.apache.spark.sql.DataFrame = [mfamily: string, mgroup: string ... 5 more fields]
trade_join_maturity: org.apache.spark.sql.DataFrame = [mfamily: string, mgroup: string ... 5 more fields]


In [5]:
// Calculo el maximo de maturity para cada origin contract number
val trade_max_maturity = trade_join_maturity.groupBy($"origin_contract_number").agg(max($"maturityReal"))

val trade_join_maturity_max = trade_join_maturity.as("a").join(trade_max_maturity.as("b"),$"a.origin_contract_number"===$"b.origin_contract_number","inner")
.drop($"b.origin_contract_number")

//trade_join_maturity_max.groupBy($"origin_contract_number").agg(count($"origin_contract_number")).show(5)

// asegurandome que se coge el mayor cuando hay duplicados con un ejemplo concreto
trade_join_maturity_max.where($"origin_contract_number"=== "19622128").show()

trade_join_maturity_max.show()

// Aquí asegurandome que no se ha perdido ningún dato hasta ahora
//trade_join_maturity_max.select(count("*")).show()

+-------+------+-----+-------------------+----------------------+----------+------------+-----------------+
|mfamily|mgroup|mtype|origin_trade_number|origin_contract_number|  maturity|maturityReal|max(maturityReal)|
+-------+------+-----+-------------------+----------------------+----------+------------+-----------------+
|   CURR|   FXD|SWLEG|           20665177|              19622128|2020-04-29|  2020-04-29|       2020-12-30|
|   CURR|   FXD|SWLEG|           20665178|              19622128|2020-12-30|  2020-12-30|       2020-12-30|
+-------+------+-----+-------------------+----------------------+----------+------------+-----------------+

+-------+------+-----+-------------------+----------------------+----------+------------+-----------------+
|mfamily|mgroup|mtype|origin_trade_number|origin_contract_number|  maturity|maturityReal|max(maturityReal)|
+-------+------+-----+-------------------+----------------------+----------+------------+-----------------+
|    IRD|  BOND| null|     

trade_max_maturity: org.apache.spark.sql.DataFrame = [origin_contract_number: string, max(maturityReal): string]
trade_join_maturity_max: org.apache.spark.sql.DataFrame = [mfamily: string, mgroup: string ... 6 more fields]


In [6]:
// Eliminar las filas cuyo maturityReal no coincide con max(maturityReal)

val trade_join_maturity_eliminados = trade_join_maturity_max.where($"maturityReal"===$"max(maturityReal)").drop("maturityReal","max(maturityReal)")

// Asegurando que tengo tantas filas como debería (26)
trade_join_maturity_eliminados.select(count("*")).show()

trade_join_maturity_eliminados.show()

+--------+
|count(1)|
+--------+
|      26|
+--------+

+-------+------+-----+-------------------+----------------------+----------+
|mfamily|mgroup|mtype|origin_trade_number|origin_contract_number|  maturity|
+-------+------+-----+-------------------+----------------------+----------+
|    IRD|  BOND| null|          316391872|             678876251|2021-09-22|
|   CURR|   FXD|  FXD|           32734782|              54853428|2021-09-22|
|    IRD| LN_BR| null|               1111|                  2222|2022-10-06|
|    IRD|   IRS| null|            2222222|               2222222|2024-10-15|
|    SCF|   SCF|  SCF|            3815982|               3672136|      NULL|
|    IRD| LN_BR| null|           14596583|              13774383|2020-12-29|
|    IRD|   IRS| null|           18343978|              17356077|2024-10-15|
|    IRD| LN_BR| null|           19203839|              18176215|2022-10-06|
|    IRD|    CF| null|           20513130|              19433281|2021-07-06|
|    IRD|    CF| nul

trade_join_maturity_eliminados: org.apache.spark.sql.DataFrame = [mfamily: string, mgroup: string ... 4 more fields]


### Resultado:

**INSTRUCCIONES**: El DataFrame resultante debe almacenarse en la variable `resultado`, sustituyendo el valor `None` por el código que consideréis oportuno. De esta forma podréis comprobar si el resultado es correcto.


In [7]:
val resultado = trade_join_maturity_eliminados

resultado: org.apache.spark.sql.DataFrame = [mfamily: string, mgroup: string ... 4 more fields]


Ejecuta la siguiente celda (no modifiques su código) y te dirá si tu solución es correcta o no. En caso de ser correcta, se ejecutará correctamente y no mostrará nada, pero si no lo es mostrará un error. Además de esas pruebas, se realizarán algunas más (ocultas) a la hora de puntuar el ejercicio, pero evaluar dicha celda es un indicador bastante fiable acerca de si realmente has implementado la solución correcta o no.

In [8]:
assert(resultado.count() == 26)
assert(resultado.orderBy("origin_contract_number").collect()(24)(4) == "564367838")
assert(resultado.orderBy("origin_contract_number").collect()(19)(5) == "NULL")
assert(resultado.orderBy("origin_trade_number").collect()(16)(5) == "NULL")